### 1. IMPORTS

In [ ]:
from pathlib import Path
import sys
import pandas as pd

### 2. ORGANIZANDO ARQUIVOS

In [ ]:
BASE_DIR = Path.cwd().parent
sys.path.append(str(BASE_DIR / "src" / "data_generation"))

from compute_financials import *

# Paths dos arquivos
DATA_RAW = BASE_DIR / "data/raw"
DATA_PROCESSED = BASE_DIR / "data/processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

producao_path = DATA_RAW / "producao_mensal.xlsx"
custos_gerais_path = DATA_RAW / "custos_gerais.xlsx"
processed_path = DATA_PROCESSED / "financials_consolidated.xlsx"

### 3. RODANDO SCRIPTS

In [ ]:
# 1. Carregar arquivos
df_prod = pd.read_excel(producao_path)
df_custos_gerais = pd.read_excel(custos_gerais_path)

In [ ]:
# 2. Aplicar share de custos gerais
df_prod = aplicar_share_custos_gerais(df_prod, df_custos_gerais)

In [ ]:
# 3. Calcular lucro líquido
df_prod = calcular_lucro_liquido(df_prod)

In [ ]:
# 4. Consolidar agregados mensais
df_agg = consolidar_agregados(df_prod)

### 4. SALVAR ARQUIVOS

In [ ]:
salvar_financeiro(df_prod, df_agg, producao_path, processed_path)

### 5. INSPEÇÃO

In [ ]:
print("Amostra de produção mensal com lucro líquido:")
display(df_prod.head())

print("Amostra de agregados mensais:")
display(df_agg.head())

### 6. CHECAGENS RAPIDAS

In [ ]:
# Margem média mensal
df_prod["margem_liquida_pct"] = df_prod["lucro_liquido_brl"] / df_prod["receita"]
margem_media = df_prod["margem_liquida_pct"].mean() * 100
print(f"\nMargem líquida média: {margem_media:.2f}%")
# Verificar se o lucro líquido total é positivo
lucro_total = df_prod["lucro_liquido_brl"].sum()
if lucro_total > 0:
    print(f"Lucro líquido total positivo: R${lucro_total:,.2f}")
else:
    print(f"Lucro líquido total negativo: R${lucro_total:,.2f}")
# Verificar o total de custos gerais alocados
total_custos_gerais = df_prod["custo_geral_alocado_brl"].sum()
print(f"Total de custos gerais alocados: R${total_custos_gerais:,.2f}")
# Verificar receita total
total_receita = df_prod["receita"].sum()
print(f"Receita total: R${total_receita:,.2f}")
# Verificar custo total
total_custo = df_prod["custo_total_brl"].sum()
print(f"Custo total: R${total_custo:,.2f}")
# Verificar se há valores nulos
null_counts = df_prod.isnull().sum()
if null_counts.any():
    print("\nAtenção: Existem valores nulos nas seguintes colunas:")
    print(null_counts[null_counts > 0])
else:
    print("\nNenhum valor nulo encontrado no DataFrame de produção.")
# Estatísticas descritivas básicas
print("\nEstatísticas descritivas do DataFrame de produção:")
print(df_prod.describe())